In [1]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
import pandas as pd
import numpy as np


In [2]:
df = pd.read_csv('test.csv',encoding='ISO-8859-1') 
print(df.shape)

(4815, 9)


In [3]:
df = df[['text', 'sentiment']]

# Clean the text data
df['text'] = df['text'].astype(str).str.lower().str.replace('[^\w\s]', '')  # Lowercase and remove special characters
df = df.dropna(subset=['text'])

<>:4: SyntaxWarning: invalid escape sequence '\w'
<>:4: SyntaxWarning: invalid escape sequence '\w'
C:\Users\hp\AppData\Local\Temp\ipykernel_23768\2242373613.py:4: SyntaxWarning: invalid escape sequence '\w'
  df['text'] = df['text'].astype(str).str.lower().str.replace('[^\w\s]', '')  # Lowercase and remove special characters


In [4]:
# Tokenization


# Step 2: Tokenization and Padding
tokenizer = Tokenizer(num_words=5000)  # Use top 5000 words
tokenizer.fit_on_texts(df['text'])  # Fit on text column
X = tokenizer.texts_to_sequences(df['text'])  # Convert text to sequences

# Padding
X = pad_sequences(X, maxlen=100)  # Pad sequences to length 100

# Step 3: Label encoding (sentiment)
label_encoder = LabelEncoder()
y = label_encoder.fit_transform(df['sentiment'])  # Convert sentiment labels to numerical

# Step 4: Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

print(X_test)


[[   0    0    0 ...  766   18  118]
 [   0    0    0 ...    1  887 2040]
 [   0    0    0 ...    0    0    2]
 ...
 [   0    0    0 ...    7 2176  127]
 [   0    0    0 ... 4024   12  833]
 [   0    0    0 ...   72   32  140]]


In [5]:

model = Sequential()
model.add(Embedding(input_dim=5000, output_dim=100, input_length=100))  # Embedding layer
model.add(LSTM(units=128, dropout=0.2, recurrent_dropout=0.2))  # LSTM layer
model.add(Dense(1, activation='sigmoid'))  # Output layer for binary classification

# Compile model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])


c:\Python312\Lib\site-packages\keras\src\layers\core\embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


In [6]:
model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ embedding (Embedding)           │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm (LSTM)                     │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ ?                      │   0 (unbuilt) │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 0 (0.00 B)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 0 (0.00 B)

In [7]:
model.fit(X_train, y_train, epochs=5, batch_size=32, validation_data=(X_test, y_test))

# Evaluate on test set
loss, accuracy = model.evaluate(X_test, y_test)

Epoch 1/5
121/121 ━━━━━━━━━━━━━━━━━━━━ 9s 49ms/step - accuracy: 0.3139 - loss: -6.2151 - val_accuracy: 0.2783 - val_loss: -15.1645
Epoch 2/5
121/121 ━━━━━━━━━━━━━━━━━━━━ 5s 45ms/step - accuracy: 0.3034 - loss: -19.4210 - val_accuracy: 0.2783 - val_loss: -23.1320
Epoch 3/5
121/121 ━━━━━━━━━━━━━━━━━━━━ 6s 46ms/step - accuracy: 0.2983 - loss: -26.5062 - val_accuracy: 0.2783 - val_loss: -31.1509
Epoch 4/5
121/121 ━━━━━━━━━━━━━━━━━━━━ 6s 49ms/step - accuracy: 0.2957 - loss: -37.7732 - val_accuracy: 0.2783 - val_loss: -38.8932
Epoch 5/5
121/121 ━━━━━━━━━━━━━━━━━━━━ 6s 48ms/step - accuracy: 0.3110 - loss: -43.6204 - val_accuracy: 0.2783 - val_loss: -46.7632
31/31 ━━━━━━━━━━━━━━━━━━━━ 1s 19ms/step - accuracy: 0.3090 - loss: -44.9436


In [8]:

print(f'Test Accuracy: {accuracy:.2f}')


Test Accuracy: 0.28
